In [1]:
from cassandra.cluster import Cluster

In [20]:
from cassandra.cluster import Cluster
import time, sys, requests
from datetime import datetime
import pandas as pd


class AlphaAPI:
    
    # Take as input: api-key, currency symbol
    
    def __init__(self, key, sym):
        self.name = 'AlphaVantage API'
        self.provider = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol='
        self.key = key
        self.sym = sym
        try:
            self.get_daily() # contains all daily data from selected currency. Created at instanciation.
        except:
            pass
        
    def get_daily(self):
        r = requests.get(self.provider+self.sym+'&outputsize=full&interval=1min&apikey='+self.key)
        content = r.json()
        all_dates = list(content['Time Series (1min)'].keys())
        timestp = [int(time.mktime(datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple())) for x in all_dates]
        vals = [content['Time Series (1min)'][x] for x in all_dates]
        data = list(zip(timestp, vals))
        self.daily_data = data
        return data
    
    def get_last(self):
        return self.daily_data[0]
    
    def insert_data(self, data):
    
        values = [float(x) for x in list(data[1].values())]
        values = ['\''+self.sym+'\'']+[data[0]]+ values
        query = 'INSERT INTO rates(company, timestp, close, high, low, open, volume) VALUES(%s, %s, %s, %s, %s, %s, %s)' % tuple(values)
        print(query)

        cluster = Cluster(['172.17.0.2', '172.17.0.3'])
        session = cluster.connect('exchange_rate')
        session.execute(query)
        session.shutdown()
        
    def get_data(self):
        cluster = Cluster(['172.17.0.2', '172.17.0.3'])
        session = cluster.connect('exchange_rate')
        return session.execute('select * from rates')
        
    def insert_daily(self):
        for item in self.daily_data:
            self.insert_data(item)
            
    def daily_to_csv(self):
        data = [[x[0]] + list(x[1].values()) for x in self.daily_data]
        df = pd.DataFrame(data)
        df.to_csv(self.sym+'_Daily.csv')
    
    def clock_stop(self):
        # bloque le script jusqu'aux heures d'ouverture des marchés
        actual = datetime.now().hour
        if actual < 9 or actual > 16:
            print('Market close time. Sleeping...')
            while True:
                try:
                    actual = datetime.now().hour
                    if actual >= 9 and actual < 16:
                        print('Market opening. Resume monitoring.')
                        break
                    time.sleep(3600)
                except KeyboardInterrupt:
                    sys.exit('Monitoring stopped.')
        
    def realtime(self, currency_list):


        while True:
            self.clock_stop()
            try:
                if type(currency_list) == list:
                    for currency in currency_list :
                        self.sym = currency
                        stocks = self.get_last()
                        self.insert_data(stocks)
                        time.sleep(10)
                    time.sleep(300)
                if type(currency_list) == str:
                    stocks = self.get_last()
                    self.insert_data(stocks)
                    time.sleep(300)
            except KeyboardInterrupt:
                sys.exit('Monitoring stopped.')
            except:
                pass
            
    def daily_monitor(self, currency_list):
        
        
        while True:
            self.clock_stop()
            try:
                if type(currency_list) == list:
                    for currency in currency_list:
                        self.get_daily()
                        self.insert_daily()
                if type(currency_list) == str:
                    self.insert_daily()
            except KeyboardInterrupt:
                sys.exit('Monitoring stopped.')
            except:
                pass

            
company = 'GOOGL'
alpha = AlphaAPI('FKFHGDZAFCX3DQQL', company)
#alpha.daily_monitor(company)
alpha.get_daily()

[(1518706800,
  {'1. open': '1091.3100',
   '2. high': '1092.8700',
   '3. low': '1090.6100',
   '4. close': '1091.3600',
   '5. volume': '173074'}),
 (1518706740,
  {'1. open': '1091.1000',
   '2. high': '1091.5900',
   '3. low': '1091.0300',
   '4. close': '1091.3400',
   '5. volume': '13941'}),
 (1518706680,
  {'1. open': '1090.7200',
   '2. high': '1090.9000',
   '3. low': '1090.3800',
   '4. close': '1090.9000',
   '5. volume': '2900'}),
 (1518706620,
  {'1. open': '1089.7600',
   '2. high': '1090.7200',
   '3. low': '1089.7100',
   '4. close': '1090.7200',
   '5. volume': '4321'}),
 (1518706560,
  {'1. open': '1089.5600',
   '2. high': '1089.7450',
   '3. low': '1089.4200',
   '4. close': '1089.7450',
   '5. volume': '3064'}),
 (1518706500,
  {'1. open': '1088.8800',
   '2. high': '1088.9700',
   '3. low': '1087.8900',
   '4. close': '1088.9600',
   '5. volume': '6420'}),
 (1518706440,
  {'1. open': '1089.1625',
   '2. high': '1089.5400',
   '3. low': '1088.8800',
   '4. close': 

In [22]:
alpha.insert_daily()

INSERT INTO rates(company, timestp, close, high, low, open, volume) VALUES('GOOGL', 1518706800, 1091.31, 1092.87, 1090.61, 1091.36, 173074.0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead


NoHostAvailable: ('Unable to connect to any servers', {'172.17.0.3': OSError(113, "Tried connecting to [('172.17.0.3', 9042)]. Last error: No route to host"), '172.17.0.2': OSError(113, "Tried connecting to [('172.17.0.2', 9042)]. Last error: No route to host")})

In [10]:
company = ['GOOGL', 'AAPL', 'FB', 'AMZN']

for comp in company:
    alpha = AlphaAPI('FKFHGDZAFCX3DQQL', comp)
    alpha.daily_to_csv()

In [33]:
## Script to transform text data to Dataframe ##
file = 'log.txt'

with open(file, 'r') as file:
    lines = file.readlines()
    file.close()
    
new = [x.strip('\n').split(',') for x in lines]
df = pd.DataFrame(new)

,0,1,2,3,4,5,6
0,GOOGL,1518015600,1056.0550,1056.0599,1055.0100,1055.4100,154899
1,AAPL,1518015600,159.6500,159.6700,159.5000,159.5400,2679808
2,FB,1518015600,180.1200,180.2000,179.9900,180.1800,1087769
3,AMZN,1518015600,1419.5600,1419.7075,1415.1500,1416.7800,230011
4,GOOGL,1518015600,1056.0550,1056.0599,1055.0100,1055.4100,154899
5,AAPL,1518015600,159.6500,159.6700,159.5000,159.5400,2679808
6,FB,1518015600,180.1200,180.2000,179.9900,180.1800,1087769
7,AMZN,1518015600,1419.5600,1419.7075,1415.1500,1416.7800,230011
8,GOOGL,1518014640,1065.6800,1065.2100,1065.9500,2100,1065.2100


In [74]:
rows = alpha.get_data()
data = []
for row in rows:
    data.append(row)
    
aslist = []
for row in data:
    l = []
    for e in range(len(row)):
        l.append(row[e])
    aslist.append(l)
    
    
df = pd.DataFrame.from_records(aslist)
print(df.shape[0], 'rows')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead


15543 rows
